<div style="text-align: center; margin-bottom: 2em;">
    <h1 style="font-size: 2.5em; margin-bottom: 0.5em; color: #2c3e50;">Machine learning algorithms for image time series</h1>
    <div style="font-size: 1.2em; color: #7f8c8d; margin-top: 1em;">
        Gilberto Camara
    </div>
</div>


### Configurations

In [ ]:
# load package "tibble"
library(tibble)
# load packages "sits" and "sitsdata"
library(sits)
library(sitsdata)
library(xgboost)
library(torch)
library(randomForest)
library(randomForestExplainer)
# set tempdir if it does not exist 
tempdir_r <- "data/th_machine_learning"
dir.create(tempdir_r, showWarnings = FALSE, recursive = TRUE)

## Data Used in This Chapter

The following examples show how to train machine learning methods and apply them to classify a single time series. We use the set `samples_matogrosso_mod13q1`, containing time series samples from the Brazilian Mato Grosso state obtained from the MODIS MOD13Q1 product. It has 1,892 samples and nine classes (`Cerrado`, `Forest`, `Pasture`, `Soy_Corn`, `Soy_Cotton`, `Soy_Fallow`, `Soy_Millet`). Each time series covers 12 months (23 data points) with six bands (NDVI, EVI, BLUE, RED, NIR, MIR). The samples are arranged along an agricultural year, starting in September and ending in August. The dataset was used in the paper "Big Earth observation time series analysis for monitoring Brazilian agriculture" [[16](#ref-picoli2018)], and is available in the R package `sitsdata`.

## Common Interface to Machine Learning and Deep Learning Models

The `sits_train()` function provides a standard interface to all machine learning models. This function takes two mandatory parameters: the training data (`samples`) and the ML algorithm (`ml_method`). After the model is estimated, it can classify individual time series or data cubes with `sits_classify()`. In what follows, we show how to apply each method to classify a single time series. Then, in Chapter [Image classification in data cubes](https://e-sensing.github.io/sitsbook/cl_rasterclassification.html), we discuss how to classify data cubes.

Since `sits` is aimed at remote sensing users who are not machine learning experts, it provides a set of default values for all classification models. These settings have been chosen based on testing by the authors. Nevertheless, users can control all parameters for each model. Novice users can rely on the default values, while experienced ones can fine-tune model parameters to meet their needs. Model tuning is discussed at the end of this Chapter. 

When a set of time series organized as tibble is taken as input to the classifier, the result is the same tibble with one additional column (`predicted`), which contains the information on the labels assigned for each interval. The results can be shown in text format using the function `sits_show_prediction()` or graphically using `plot()`.

## Random Forest

Random Forest is a machine learning algorithm that uses an ensemble learning method for classification tasks. The algorithm consists of multiple decision trees, each trained on a different subset of the training data and with a different subset of features. To make a prediction, each decision tree in the forest independently classifies the input data. The final prediction is made based on the majority vote of all the decision trees. The randomness in the algorithm comes from the random subsets of data and features used to train each decision tree, which helps to reduce overfitting and improve the accuracy of the model. This classifier measures the importance of each feature in the classification task, which can be helpful in feature selection and data visualization. For an in-depth discussion of the robustness of Random Forest for satellite image time series classification, please see Pelletier et al [[14](#ref-pelletier2016)]. 

<a id="fig-ml-rf"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/random_forest.png" alt="random_forest" style="width:80%">
    <figcaption align="center">Figure 1: random_forest</figcaption>
    </figure>
</center>

`sits` provides `sits_rfor()`, which uses the R `randomForest` package [[25](#ref-wright2017)]; its main parameter is `num_trees`, which is the number of trees to grow with a default value of 100. The model can be visualized using `plot()`.

In [ ]:
set.seed(290356)

# Train the Mato Grosso samples with Random Forest algorithm
rfor_model <- sits_train(
    samples = samples_matogrosso_mod13q1, 
    ml_method = sits_rfor(num_trees = 100))

# Plot the most important variables of the model
plot(rfor_model)

<a id="fig-ml-rfimp"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mlrformodel.png" alt="mlrformodel" style="width:80%">
    <figcaption align="center">Figure 2: mlrformodel</figcaption>
    </figure>
</center>

The model plot shows the most important explanatory variables, which are the NIR (near infrared) band on date 17 (2007-05-25) and the MIR (middle infrared) band on date 22 (2007-08-13). The NIR value at the end of May captures the growth of the second crop for double cropping classes.  Values of the MIR band at the end of the period (late July to late August) capture bare soil signatures to distinguish between agricultural and natural classes. This corresponds to summertime when the ground is drier after harvesting crops.

In [ ]:
# Classify using Random Forest model and plot the result
point_class <- sits_classify(
    data = point_mt_mod13q1, 
    ml_model  = rfor_model)

plot(point_class, bands = c("NDVI", "EVI"))

<a id="fig-ml-rfplot"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mlrforplot.png" alt="mlrforplot" style="width:80%">
    <figcaption align="center">Figure 3: mlrforplot</figcaption>
    </figure>
</center>

The result shows that the area started as a forest in 2000, was deforested from 2004 to 2005, used as pasture from 2006 to 2007, and for double-cropping agriculture from 2009 onwards. This behavior is consistent with expert evaluation of land change process in this region of Amazonia.

Random Forest is robust to outliers and can deal with irrelevant inputs [[9](#ref-hastie2009)]. The method tends to overemphasize some variables because its performance tends to stabilize after part of the trees is grown [[9](#ref-hastie2009)]. In cases where abrupt change occurs, such as deforestation mapping, Random Forest (if properly trained) will emphasize the temporal instances and bands that capture such quick change. Before using Random Forest, it is recommended that users balance their training samples as explained in ["Reducing imbalances in training samples"](https://e-sensing.github.io/sitsbook/cl_rasterclassification.html)

## Extreme Gradient Boosting

XGBoost (eXtreme Gradient Boosting) [[3](#ref-chen2016)] is an implementation of gradient boosted decision trees designed for speed and performance.  It is an ensemble learning method, meaning it combines the predictions from multiple models to produce a final prediction. XGBoost builds trees one at a time, where each new tree helps to correct errors made by previously trained tree. Each tree builds a new model to correct the errors made by previous models. Using gradient descent, the algorithm iteratively adjusts the predictions of each tree by focusing on instances where previous trees made errors. Models are added sequentially until no further improvements can be made. 

Although Random Forest and boosting use trees for classification, there are significant differences. While Random Forest builds multiple decision trees in parallel and merges them together later, XGBoost builds trees one at a time. In XGBoost, each new tree helps to correct errors made by previously trained tree. XGBoost is often preferred for its speed and performance, particularly on large datasets and is well-suited for problems where precision is paramount. Random Forest, on the other hand, is simpler to implement, more interpretable, and can be more robust to overfitting, making it a good choice for general-purpose applications.

<a id="fig-ml-xgb"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/flow_chart_xgboost.png" alt="flow_chart_xgboost" style="width:80%">
    <figcaption align="center">Figure 4: flow_chart_xgboost</figcaption>
    </figure>
</center>

The boosting method starts from a weak predictor and then improves performance sequentially by fitting a better model at each iteration. It fits a simple classifier to the training data and uses the residuals of the fit to build a predictor. Typically, the base classifier is a regression tree. Although random forest and boosting use trees for classification, there are significant differences. The performance of Random Forest generally increases with the number of trees until it becomes stable. Boosting trees apply finer divisions over previous results to improve performance [[9](#ref-hastie2009)]. Some recent papers show that it outperforms Random Forest for remote sensing image classification [[12](#ref-jafarzadeh2021)]. However, this result is not generalizable since the quality of the training dataset controls actual performance.

In `sits`, the XGBoost method is implemented by the `sits_xbgoost()` function, based on `XGBoost` R package, and has five hyperparameters that require tuning. The `sits_xbgoost()` function takes the user choices as input to a cross-validation to determine suitable values for the predictor.

The learning rate `eta` varies from 0.0 to 1.0 and should be kept small (default is 0.3) to avoid overfitting. The minimum loss value `gamma` specifies the minimum reduction required to make a split. Its default is 0; increasing it makes the algorithm more conservative. The `max_depth` value controls the maximum depth of the trees. Increasing this value will make the model more complex and likely to overfit (default is 6). The `subsample` parameter controls the percentage of samples supplied to a tree. Its default is 1 (maximum). Setting it to lower values means that xgboost randomly collects only part of the data instances to grow trees, thus preventing overfitting. The `nrounds` parameter controls the maximum number of boosting interactions; its default is 100, which has proven to be enough in most cases. To follow the convergence of the algorithm, users can turn the `verbose` parameter on. In general, the results using the extreme gradient boosting algorithm are similar to the Random Forest method.


In [ ]:
set.seed(290356)

# Train using  XGBoost
xgb_model <- sits_train(
    samples = samples_matogrosso_mod13q1, 
    ml_method = sits_xgboost(verbose = FALSE))

# Classify using SVM model and plot the result
point_class_xgb <- sits_classify(
    data = point_mt_mod13q1, 
    ml_model = xgb_model)

# View classification
plot(point_class_xgb, bands = c("NDVI", "EVI"))

<a id="fig-ml-xgbplot"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mlxgbplot.png" alt="mlxgbplot" style="width:80%">
    <figcaption align="center">Figure 5: mlxgbplot</figcaption>
    </figure>
</center>

## Deep Learning using Multilayer Perceptrons

To support deep learning methods, `sits` uses the `torch` R package, which takes the Facebook `torch` C++ library as a back-end. Machine learning algorithms that use the R `torch` package are similar to those developed using `PyTorch`. The simplest deep learning method is multilayer perceptron (MLP), which are feedforward artificial neural networks. An MLP consists of three kinds of nodes: an input layer, a set of hidden layers, and an output layer. The input layer has the same dimension as the number of features in the dataset. The hidden layers attempt to approximate the best classification function. The output layer decides which class should be assigned to the input. In an MLP, all inputs are treated equally at first; based on iterative matching of training and test data, the backpropagation technique feeds information back to the initial layers to identify the most suitable combination of inputs that produces the best output.

In `sits`, MLP models can be built using `sits_mlp()`. Since there is no established model for generic classification of satellite image time series, designing MLP models requires parameter customization. The most important decisions are the number of layers in the model and the number of neurons per layer. These values are set by the `layers` parameter, which is a list of integer values. The size of the list is the number of layers, and each element indicates the number of nodes per layer.

The choice of the number of layers depends on the inherent separability of the dataset to be classified. For datasets where the classes have different signatures, a shallow model (with three layers) may provide appropriate responses. More complex situations require models of deeper hierarchy. Models with many hidden layers may take a long time to train and may not converge. We suggest to start with three layers and test different options for the number of neurons per layer before increasing the number of layers. In our experience, using three to five layers is a reasonable compromise if the training data has a good quality. Further increase in the number of layers will not improve the model. 

MLP models also need to include the activation function. The activation function of a node defines the output of that node given an input or set of inputs. Following standard practices [[8](#ref-goodfellow2016)], we use the `relu` activation function.

The optimization method (`optimizer`) represents the gradient descent algorithm to be used. These methods aim to maximize an objective function by updating the parameters in the opposite direction of the gradient of the objective function [[17](#ref-ruder2016)]. Since gradient descent plays a key role in deep learning model fitting, developing optimizers is an important topic of research [[1](#ref-bottou2018)]. Many optimizers have been proposed in the literature, and recent results are reviewed by Schmidt et al. [[20](#ref-schmidt2021)]. The Adamw optimizer  provides a good baseline and reliable performance for general deep learning applications [[13](#ref-kingma2017)]. By default, all deep learning algorithms in `sits` use Adamw. 

Another relevant parameter is the list of dropout rates (`dropout`). Dropout is a technique for randomly dropping units from the neural network during training [[22](#ref-srivastava2014)]. By randomly discarding some neurons, dropout reduces overfitting. Since a cascade of neural nets aims to improve learning as more data is acquired, discarding some neurons may seem like a waste of resources. In practice, dropout prevents an early convergence to a local minimum [[8](#ref-goodfellow2016)]. We suggest users experiment with different dropout rates, starting from small values (10-30%) and increasing as required.

The following example shows how to use `sits_mlp()`. The default parameters have been chosen based on a modified version of [24](#ref-wang2017), which proposes using multilayer perceptron as a baseline for time series classification. These parameters are: (a) Three layers with 512 neurons each, specified by the parameter `layers`; (b) Using the "relu" activation function; (c) dropout rates of 40%, 30%, and 20% for the layers; (d) the "optimizer_adamw" as optimizer (default value); (e) a number of training steps (`epochs`) of 100; (f) a `batch_size` of 64, which indicates how many time series are used for input at a given step; and (g) a validation percentage of 20%, which means 20% of the samples will be randomly set aside for validation.

To simplify the output, the `verbose` option has been turned off. After the model has been generated, we plot its training history.

In [ ]:
set.seed(290356)

# Train using an MLP model
# This is an example of how to set parameters
# First-time users should test default options first
mlp_model <- sits_train(
    samples = samples_matogrosso_mod13q1, 
    ml_method = sits_mlp(
        optimizer        = torch::optim_adamw, 
        layers           = c(512, 512, 512),
        dropout_rates    = c(0.40, 0.30, 0.20),
        epochs           = 80,
        batch_size       = 64,
        verbose          = FALSE,
        validation_split = 0.2))

# Show training evolution
plot(mlp_model)

<a id="fig-ml-mlp"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mlmlpmodel.png" alt="mlmlpmodel" style="width:80%">
    <figcaption align="center">Figure 6: mlmlpmodel</figcaption>
    </figure>
</center>

Then, we classify a 16-year time series using the multilayer perceptron model.

In [ ]:
# Classify using MLP model and plot the result
point_mt_mod13q1 |>  
    sits_classify(mlp_model) |>  
    plot(bands = c("NDVI", "EVI"))

<a id="fig-ml-mlplot"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mlmlpplot.png" alt="mlmlpplot" style="width:80%">
    <figcaption align="center">Figure 7: mlmlpplot</figcaption>
    </figure>
</center>

In theory, multilayer perceptron model can capture more subtle changes than Random Forest and XGBoost. In this specific case, the result is similar to theirs. Although the model mixes the `Soy_Corn` and `Soy_Millet` classes, the distinction between their temporal signatures is quite subtle. Also it suggests the need to improve the number of samples. In this example, the MLP model shows an increase in sensitivity compared to previous models. We recommend to compare different configurations since the MLP model is sensitive to changes in its parameters.

## Temporal Convolutional Neural Network (TempCNN)

Convolutional neural networks (CNN) are deep learning methods that apply convolution filters (sliding windows) to the input data sequentially. The Temporal Convolutional Neural Network (TempCNN) is a neural network architecture specifically designed to process sequential data such as time series. In the case of time series, a 1D CNN applies a moving temporal window to the time series to produce another time series as the result of the convolution. 

The TempCNN architecture for satellite image time series classification is proposed by Pelletier et al. [[15](#ref-pelletier2019)].  It has three 1D convolutional layers and a final softmax layer for classification. The authors combine different methods to avoid overfitting and reduce the vanishing gradient effect, including dropout, regularization, and batch normalization. In the TempCNN reference paper [[15](#ref-pelletier2019)], the authors favourably compare their model with the Recurrent Neural Network proposed by Russwurm and Körner [[18](#ref-russwurm2018)]. [Figure 8](#fig-ml-tcnn) shows the architecture of the TempCNN model. TempCNN applies one-dimensional convolutions on the input sequence to capture temporal dependencies, allowing the network to learn long-term dependencies in the input sequence. Each layer of the model captures temporal dependencies at a different scale. Due to its multi-scale approach, TempCNN can capture complex temporal patterns in the data and produce accurate predictions.

<a id="fig-ml-tcnn"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/tempcnn.png" alt="tempcnn" style="width:80%">
    <figcaption align="center">Figure 8: tempcnn</figcaption>
    </figure>
</center>

The function `sits_tempcnn()` implements the model. The first parameter is the `optimizer` used in the backpropagation phase for gradient descent. The default is `adamw` which is considered as a stable and reliable optimization function. The parameter `cnn_layers` controls the number of 1D-CNN layers and the size of the filters applied at each layer; the default values are three CNNs with 128 units. The parameter `cnn_kernels` indicates the size of the convolution kernels; the default is kernels of size 7. Activation for all 1D-CNN layers uses the "relu" function. The dropout rates for each 1D-CNN layer are controlled individually by the parameter `cnn_dropout_rates`. The `validation_split` controls the size of the test set relative to the full dataset. We recommend setting aside at least 20% of the samples for validation.

In [ ]:
set.seed(290356)
library(torch)

# Train using tempCNN
tempcnn_model <- sits_train(
    samples_matogrosso_mod13q1, 
    sits_tempcnn(
        optimizer            = torch::optim_adamw,
        cnn_layers           = c(256, 256, 256),
        cnn_kernels          = c(7, 7, 7),
        cnn_dropout_rates    = c(0.2, 0.2, 0.2),
        epochs               = 80,
        batch_size           = 64,
        validation_split     = 0.2,
        verbose              = TRUE))

# Show training evolution
plot(tempcnn_model)

<a id="fig-ml-tcnnmodel"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mltcnnmodel.png" alt="mltcnnmodel" style="width:80%">
    <figcaption align="center">Figure 9: mltcnnmodel</figcaption>
    </figure>
</center>

Using the TempCNN model, we classify a 16-year time series.

In [ ]:
# Classify using TempCNN model and plot the result
point_mt_mod13q1 |>  
    sits_classify(tempcnn_model) |>  
    plot(bands = c("NDVI", "EVI"))

<a id="fig-ml-tcnnplot"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mltcnnplot.png" alt="mltcnnplot" style="width:80%">
    <figcaption align="center">Figure 10: mltcnnplot</figcaption>
    </figure>
</center>

The result has important differences from the previous ones. The TempCNN model indicates the `Soy_Cotton` class as the most likely one in 2004. While this result is possibly wrong, it shows that the time series for 2004 is different from those of Forest and Pasture classes. One possible explanation is that there was forest degradation in 2004, leading to a signature that is a mix of forest and bare soil. In this case, including forest degradation samples could improve the training data. In our experience, TempCNN models are a reliable way of classifying image time series [[21](#ref-simoes2021)]. Recent work which compares different models also provides evidence that TempCNN models have satisfactory behavior, especially in the case of crop classes [[19](#ref-russwurm2020)].

## Residual 1D CNN networks (ResNet)

A residual 1D CNN network, also known as ResNet, is an extension of the standard 1D CNN architecture,  adding residual connections between the layers. Residual connections allow the network to learn residual mappings, which are the difference between the input and output of a layer. By adding these residual connections, the network can learn to bypass specific layers and still capture essential features in the data.

The Residual Network (ResNet) for time series classification was proposed by Wang et al. [[24](#ref-wang2017)], based on the idea of deep residual networks for 2D image recognition [[10](#ref-he2016)]. The ResNet architecture comprises 11 layers, with three blocks of three 1D CNN layers each (see [Figure 11](#fig-ml-rnet)). Each block corresponds to a 1D CNN architecture. The output of each block is combined with a shortcut that links its output to its input, called a skip connection. The purpose of combining the input layer of each block with its output layer (after the convolutions) is to avoid the so-called "vanishing gradient problem". This issue occurs in deep networks as the neural network's weights are updated based on the partial derivative of the error function. If the gradient is too small, the weights will not be updated, stopping the training [[11](#ref-hochreiter1998)]. Skip connections aim to avoid vanishing gradients from occurring, allowing deep networks to be trained.

<a id="fig-ml-rnet"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/resnet.png" alt="resnet" style="width:80%">
    <figcaption align="center">Figure 11: resnet</figcaption>
    </figure>
</center>

In `sits`, the Residual Network is implemented using `sits_resnet()`. The default parameters are those proposed by Wang et al. [[24](#ref-wang2017)], as implemented by Fawaz et al. [[5](#ref-fawaz2019)]. The first parameter is `blocks`, which controls the number of blocks and the size of filters in each block. By default, the model implements three blocks, the first with 64 filters and the others with 128. The parameter `kernels` controls the size of the kernels of the three layers inside each block. It is useful to experiment a bit with these kernel sizes in the case of satellite image time series. The default activation is "relu", which is recommended in the literature to reduce the problem of vanishing gradients. The default optimizer is `optim_adamw`, available in package `torchopt`.

In [ ]:
# Train using ResNet
resnet_model <- sits_train(samples_matogrosso_mod13q1, 
                       sits_resnet(
                          blocks               = c(64, 128, 128),
                          kernels              = c(7, 5, 3),
                          epochs               = 100,
                          batch_size           = 64,
                          validation_split     = 0.2,
                          verbose              = FALSE))
# Show training evolution
plot(resnet_model)

<a id="fig-ml-resnetmodel"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mlresnetmodel.png" alt="mlresnetmodel" style="width:80%">
    <figcaption align="center">Figure 12: mlresnetmodel</figcaption>
    </figure>
</center>

Using the TempCNN model, we classify a 16-year time series.

In [ ]:
# Classify using Resnet model and plot the result
point_mt_mod13q1 |>  
    sits_classify(resnet_model) |>  
    plot(bands = c("NDVI", "EVI"))

<a id="fig-ml-resnetplot"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mlresnetplot.png" alt="mlresnetplot" style="width:80%">
    <figcaption align="center">Figure 13: mlresnetplot</figcaption>
    </figure>
</center>

In this case, the result of the ResNet model is quite similar to that of the TempCNN model. In the same way as TempCNN, the ResNet algorithm tends to be better at detecting agricultural or natural forest classes. When it comes to situation where transitions happen during the classification period, as in the case of the transition from forest to pasture in 2004, random forest models tend to be more efficient than TempCNN or ResNet. 

## Attention-based models

Attention-based deep learning models are a class of models that use a mechanism inspired by human attention to focus on specific parts of input during processing. These models have been shown to be effective for various tasks such as machine translation, image captioning, and speech recognition.

The basic idea behind attention-based models is to allow the model to selectively focus on different input parts at different times. This is done by introducing a mechanism that assigns weights to each element of the input, indicating the relative importance of that element to the current processing step. The model uses them to compute a weighted sum of the input. The results capture the model's attention on specific parts of the input.

Attention-based models have become one of the most used deep learning architectures for problems that involve sequential data inputs, e.g., text recognition and automatic translation. The general idea is that not all inputs are alike in applications such as language translation. Consider the English sentence "Look at all the lonely people". A sound translation system needs to relate the words "look" and "people" as the key parts of this sentence to ensure such link is captured in the translation. A specific type of attention models, called transformers, enables the recognition of such complex relationships between input and output sequences [[23](#ref-vaswani2017)]. 

The basic structure of transformers is the same as other neural network algorithms. They have an encoder that transforms textual input values into numerical vectors and a decoder that processes these vectors to provide suitable answers. The difference is how the values are handled internally. The two main differences between transformer models and other algorithms are positional encoding and self-attention. Positional encoding assigns an index to each input value, ensuring that the relative locations of the inputs are maintained throughout the learning and processing phases. Self-attention compares every word in a sentence to every other word in the same sentence, including itself. In this way, it learns contextual information about the relation between the words. This conception has been validated in large language models such as BERT [[4](#ref-devlin2019)] and GPT-4 [[2](#ref-bubeck2023)].

The application of attention-based models for satellite image time series analysis is proposed by Garnot et al. [[7](#ref-garnot2020a)] and Russwurm and Körner [[19](#ref-russwurm2020)]. A self-attention network can learn to focus on specific time steps and image features most relevant for distinguishing between different classes. The algorithm tries to identify which combination of individual temporal observations is most relevant to identify each class. For example, crop identification will use observations that capture the onset of the growing season, the date of maximum growth, and the end of the growing season. In the case of deforestation, the algorithm tries to identify the dates when the forest is being cut. Attention-based models are a means to identify events that characterize each land class.

The first model proposed by Garnot et al. is a full transformer-based model [[7](#ref-garnot2020a)]. Considering that image time series classification is easier than natural language processing, Garnot et al. also propose a simplified version of the full transformer model [[6](#ref-garnot2020)]. This simpler model uses a reduced way to compute the attention matrix, reducing time for training and classification without loss of quality of the result. 

In `sits`, the full transformer-based model proposed by Garnot et al. [[7](#ref-garnot2020a)]   is implemented using `sits_tae()`. The default parameters are those proposed by the authors. The default optimizer is `optim_adamw`, as also used in the `sits_tempcnn()` function.

In [ ]:
# Train a machine learning model using TAE
tae_model <- sits_train(samples_matogrosso_mod13q1, 
                       sits_tae(
                          epochs               = 80,
                          batch_size           = 64,
                          optimizer            = torch::optim_adamw,
                          validation_split     = 0.2,
                          verbose              = FALSE))

# Show training evolution
plot(tae_model)

<a id="fig-ml-taemodel"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mltaemodel.png" alt="mltaemodel" style="width:80%">
    <figcaption align="center">Figure 14: mltaemodel</figcaption>
    </figure>
</center>

Then, we classify a 16-year time series using the TAE model. 


In [ ]:
# Classify using DL model and plot the result
point_mt_mod13q1 |>  
    sits_classify(tae_model) |>  
    plot(bands = c("NDVI", "EVI"))

<a id="fig-ml-taeplot"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mltaeplot.png" alt="mltaeplot" style="width:80%">
    <figcaption align="center">Figure 15: mltaeplot</figcaption>
    </figure>
</center>

Garnot and co-authors also proposed the Lightweight Temporal Self-Attention Encoder (LTAE) [[6](#ref-garnot2020)], which the authors claim can achieve high classification accuracy with fewer parameters compared to other neural network models. It is a good choice for applications where computational resources are limited. The `sits_lighttae()` function implements this algorithm. The most important parameter to be set is the learning rate `lr`. Values ranging from 0.001 to 0.005 should produce good results. See also the section below on model tuning. 

In [ ]:
# Train a machine learning model using TAE
ltae_model <- sits_train(samples_matogrosso_mod13q1, 
                       sits_lighttae(
                          epochs               = 80,
                          batch_size           = 64,
                          optimizer            = torch::optim_adamw,
                          opt_hparams = list(lr = 0.001),
                          validation_split     = 0.2))

# Show training evolution
plot(ltae_model)

<a id="fig-ml-ltaemodel"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mlltaemodel.png" alt="mlltaemodel" style="width:80%">
    <figcaption align="center">Figure 16: mlltaemodel</figcaption>
    </figure>
</center>

Then, we classify a 16-year time series using the LightTAE model. 

In [ ]:
# Classify using DL model and plot the result
point_mt_mod13q1 |>  
    sits_classify(ltae_model) |>  
    plot(bands = c("NDVI", "EVI"))

<a id="fig-ml-ltaeplot"></a>
<center>
    <figure>
    <img src="./images/th_machine_learning/mlltaeplot.png" alt="mlltaeplot" style="width:80%">
    <figcaption align="center">Figure 17: mlltaeplot</figcaption>
    </figure>
</center>

The behaviour of both `sits_tae()` and `sits_lighttae()` is similar to that of `sits_tempcnn()`. It points out the possible need for more classes and training data to better represent the transition period between 2004 and 2010. One possibility is that the training data associated with the Pasture class is only consistent with the time series between the years 2005 to 2008. However, the transition from Forest to Pasture in 2004 and from Pasture to Agriculture in 2009-2010 is subject to uncertainty since the classifiers do not agree on the resulting classes. In general, deep learning temporal-aware models are more sensitive to class variability than Random Forest and extreme gradient boosters. 

## Summary

In this chapter, we present a basic description of the machine learning algorithms for image time series. These methods are available in the R `sits` package, but can be used in other environments as well. The basic distinction is between time-sensitive algorithms such as LightTAE and TempCNN and those who do not consider temporal order of values, such as Random Forest. In practice, we suggest that users take Random Forest as their baseline and then use LightTAE or TempCNN to try to improve classification accuracy. 

## References


<a id="ref-bottou2018"></a>
[1] Léon Bottou, Frank E. Curtis, and Jorge Nocedal. Optimization Methods for Large-Scale Machine Learning. SIAM Review, 60(2):223–311, 2018. doi:10.1137/16M1080173.

<a id="ref-bubeck2023"></a>
[2] Sébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, Johannes Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Yin Tat Lee, Yuanzhi Li, Scott Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, and Yi Zhang. Sparks of Artificial General Intelligence: Early experiments with GPT-4. 2023. arXiv:2303.12712, doi:10.48550/arXiv.2303.12712.

<a id="ref-chen2016"></a>
[3] Tianqi Chen and Carlos Guestrin. XGBoost: A Scalable Tree Boosting System. In Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, KDD '16, 785–794. New York, NY, USA, 2016. Association for Computing Machinery. doi:10.1145/2939672.2939785.

<a id="ref-devlin2019"></a>
[4] Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. 2019. arXiv:1810.04805, doi:10.48550/arXiv.1810.04805.

<a id="ref-fawaz2019"></a>
[5] Hassan Ismail Fawaz, Germain Forestier, Jonathan Weber, Lhassane Idoumghar, and Pierre-Alain Muller. Deep learning for time series classification: a review. Data Mining and Knowledge Discovery, 33(4):917–963, 2019.

<a id="ref-garnot2020"></a>
[6] Vivien Garnot and Loic Landrieu. Lightweight Temporal Self-attention for~Classifying Satellite Images Time~Series. In Vincent Lemaire, Simon Malinowski, Anthony Bagnall, Thomas Guyet, Romain Tavenard, and Georgiana Ifrim, editors, Advanced Analytics and Learning on Temporal Data, Lecture Notes in Computer Science, 171–181. Cham, 2020. Springer International Publishing.

<a id="ref-garnot2020a"></a>
[7] Vivien Garnot, Sebastien Giordano, and Nesrine Chehata. Satellite Image Time Series Classification With Pixel-Set Encoders and Temporal Self-Attention. In 2020 IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR), 12322–12331. Seattle, WA, USA, 2020. IEEE. doi:10.1109/CVPR42600.2020.01234.

<a id="ref-goodfellow2016"></a>
[8] Ian Goodfellow, Yoshua Bengio, and Aaron Courville. Deep Learning. MIT Press, 2016.

<a id="ref-hastie2009"></a>
[9] T. Hastie, R. Tibshirani, and J Friedman. The Elements of Statistical Learning. Data Mining, Inference, and Prediction. Springer, New York, 2009.

<a id="ref-he2016"></a>
[10] K. He and others. Deep residual learning for image recognition. Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition, pages 770–778, 2016.

<a id="ref-hochreiter1998"></a>
[11] Sepp Hochreiter. The vanishing gradient problem during learning recurrent neural nets and problem solutions. International Journal of Uncertainty, Fuzziness and Knowledge-Based Systems, 6(2):107–116, 1998. doi:10.1142/S0218488598000094.

<a id="ref-jafarzadeh2021"></a>
[12] Hamid Jafarzadeh, Masoud Mahdianpari, Eric Gill, Fariba Mohammadimanesh, and Saeid Homayouni. Bagging and Boosting Ensemble Classifiers for Classification of Multispectral, Hyperspectral and PolSAR Data: A Comparative Evaluation. Remote Sensing, 13(21):4405, 2021. doi:10.3390/rs13214405.

<a id="ref-kingma2017"></a>
[13] Diederik P. Kingma and Jimmy Ba. Adam: A Method for Stochastic Optimization. 2017. arXiv:1412.6980, doi:10.48550/arXiv.1412.6980.

<a id="ref-pelletier2016"></a>
[14] Charlotte Pelletier, Silvia Valero, Jordi Inglada, Nicolas Champion, and Gerard Dedieu. Assessing the robustness of Random Forests to map land cover with high resolution satellite image time series over large areas. Remote Sensing of Environment, 187:156–168, 2016. doi:10.1016/j.rse.2016.10.010.

<a id="ref-pelletier2019"></a>
[15] Charlotte Pelletier, Geoffrey I. Webb, and Francois Petitjean. Temporal Convolutional Neural Network for the Classification of Satellite Image Time Series. Remote Sensing, 2019.

<a id="ref-picoli2018"></a>
[16] Michelle Picoli, Gilberto Camara, Ieda Sanches, Rolf Simoes, Alexandre Carvalho, Adeline Maciel, Alexandre Coutinho, Julio Esquerdo, Joao Antunes, Rodrigo Anzolin Begotti, Damien Arvor, and Claudio Almeida. Big earth observation time series analysis for monitoring Brazilian agriculture. ISPRS Journal of Photogrammetry and Remote Sensing, 145:328–339, 2018. doi:10.1016/j.isprsjprs.2018.08.007.

<a id="ref-ruder2016"></a>
[17] Sebastian Ruder. An overview of gradient descent optimization algorithms. CoRR, 2016.

<a id="ref-russwurm2018"></a>
[18] Marc Russwurm and Marco Korner. Multi-temporal land cover classification with sequential recurrent encoders. ISPRS International Journal of Geo-Information, 7(4):129, 2018.

<a id="ref-russwurm2020"></a>
[19] M. Russwurm and M. Korner. Self-attention for raw optical satellite time series classification. ISPRS Journal of Photogrammetry and Remote Sensing, 169:1–12, 2020.

<a id="ref-schmidt2021"></a>
[20] Robin M. Schmidt, Frank Schneider, and Philipp Hennig. Descending through a Crowded Valley - Benchmarking Deep Learning Optimizers. In Proceedings of the 38th International Conference on Machine Learning, 9367–9376. PMLR, 2021.

<a id="ref-simoes2021"></a>
[21] Rolf Simoes, Gilberto Camara, Gilberto Queiroz, Felipe Souza, Pedro R. Andrade, Lorena Santos, Alexandre Carvalho, and Karine Ferreira. Satellite image time series analysis for big earth observation data. Remote Sensing, 13:2428, 2021. doi:10.3390/rs13132428.

<a id="ref-srivastava2014"></a>
[22] Nitish Srivastava, Geoffrey Hinton, Alex Krizhevsky, Ilya Sutskever, and Ruslan Salakhutdinov. Dropout: A simple way to prevent neural networks from overfitting. The Journal of Machine Learning Research, 15(1):1929–1958, 2014.

<a id="ref-vaswani2017"></a>
[23] Vaswani Ashish. Attention is all you need. Advances in neural information processing systems, 30:I, 2017.

<a id="ref-wang2017"></a>
[24] Zhiguang Wang, Weizhong Yan, and Tim Oates. Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline. In 2017 International Joint Conference on Neural Networks (IJCNN). 2017.

<a id="ref-wright2017"></a>
[25] Jonathon S. Wright, Rong Fu, John R. Worden, Sudip Chakraborty, Nicholas E. Clinton, Camille Risi, Ying Sun, and Lei Yin. Rainforest-initiated wet season onset over the southern Amazon. Proceedings of the National Academy of Sciences, 2017. doi:10.1073/pnas.1621516114.

